# Investigation of geophysical sensor data to inform priors

Since we don't have a really great idea of what constitutes a good set of priors for real data, here I try my best to sort out what is going on using what I hope will be simple, but robust, assumptions.

In [ ]:
import GPy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyproj
%matplotlib inline

## Noise and length scale characteristics for gravity and magnetism

We've been running with some set of priors for gravity and magnetism, but in all fairness we have no idea what those should be.  We know they're both linear sensors that integrate over rock properties, with a 3-D sensitivity profile that gets broader with depth.  So by fitting a GP to them, we get some idea of the noise, and a lower limit on the relevant length scale.  Since they're on a grid, we could also consider the autocorrelation.

This isn't really meant to be a Bayesian analysis, but it's meant to give us some idea of the order of magnitude of the noise in a model that's flexible enough to respond to changes, but that insists on smoothness so we can pick off the delta-function component of the covariance.

# Data dictionaries

In [ ]:
length_grav = 0.05
length_mag = 0.015
length_mag_metres = 1500
lat_centre = 24.85
lon_centre = 116.1
degree_conversion = 111111

gda94 = pyproj.Proj(init='epsg:4283')
mgaz50 = pyproj.Proj(init='epsg:28350')

In [ ]:
eastings_centre, northings_centre = pyproj.transform(gda94, mgaz50, lon_centre, -lat_centre)
print(eastings_centre, northings_centre)

In [ ]:
eastings_boundary1, northings_boundary1 = pyproj.transform(gda94, mgaz50, lon_centre, -lat_centre + length_mag)
print(eastings_boundary1, northings_boundary1)

In [ ]:
print(eastings_boundary1 - eastings_centre, northings_boundary1 - northings_centre)

In [ ]:
eastings_boundary2, northings_boundary2 = pyproj.transform(gda94, mgaz50, lon_centre + length_mag, -lat_centre)
print(eastings_boundary2, northings_boundary2)

In [ ]:
print(eastings_boundary2 - eastings_centre, northings_boundary2 - northings_centre)

In [ ]:
# note: latitude is negative, longitude is positive
lat_check_func = lambda lat, lat_centre, length: np.abs(lat + lat_centre) < length
lon_check_func = lambda lon, lat_centre, length: np.abs(lon - lon_centre) < length

In [ ]:
dict_data_set = dict_data_set2
sub_key = 'mag'
length = length_mag

fpath_data, key_lat, key_lon, key_y = get_vars(dict_data_set, sub_key)
data = get_data(fpath_data)

In [ ]:
x = data['X'][0]
y = data['Y'][0]
print(x, y)

In [ ]:
print(eastings_centre, northings_centre)

In [ ]:
print(np.abs(x - eastings_centre), np.abs(y - northings_centre))

In [ ]:
degree_conversion * length_mag

In [ ]:
relevant_y_bool = np.abs(data['Y'] - northings_centre) < (degree_conversion * length_mag)
print(relevant_y_bool.sum())

In [ ]:
relevant_x_bool = np.abs(data['X'] - eastings_centre) < np.cos(data['X']) * (degree_conversion * length_mag)
print(relevant_x_bool.sum())

In [ ]:
relevant_xy_bool = relevant_y_bool & relevant_x_bool
print(relevant_xy_bool.sum())

In [ ]:
dict_data_set1 = {
    'dir_data': '/Users/davidkohn/dev/obsidian/data/dataset1',
    'grav': {
        'fname': 'gravity_400m_Gascoyne.txt',
        'key_lat': 'Latitude',
        'key_lon': 'Longitude',
        'key_y': 'grid_code',
    },
    'mag': {
        'fname': 'mag_TMI_gascoyne.txt',
        'key_lat': 'Latitude',
        'key_lon': 'Longitude',
        'key_y': 'grid_code',
    },
}

dict_data_set2 = {
    'dir_data': '/Users/davidkohn/dev/obsidian/data/dataset2',
    'grav_north': {
        'fname': 'Gascoyne_North_Bouguer_gravity_400m_XYZ.txt',
        'key_lat': 'X',
        'key_lon': 'Y',
        'key_y': 'GASCOYNE_NORTH_1',
    },
    'grav_south': {
        'fname': 'Gascoyne_South_Bouguer_gravity_500m_XYZ.txt',
        'key_lat': '',
        'key_lon': '',
        'key_y': 'GASCOYNE_SOUTH_1',
    },
    'mag': {
        'fname': 'Bangemall_mag_125m_XYZ.txt',
        'key_lat': '',
        'key_lon': '',
        'key_y': 'MAG_PD',
    },
}

# Functions

In [ ]:
msg_str0 = 'fpath: {}'
msg_str1 = '  Latitude min: {}\n  Latitude max: {}'
msg_str2 = '  Grid code min: {}\n  Grid code max: {}'
msg_str3 = '  Data shape: {}'
msg_str4 = '  X shape: {}\n  Y shape: {}'
msg_str5 = '  Data columns: {}'

def get_vars(dict_data_set, sub_key):
    dir_data = dict_data_set['dir_data']
    fname_data = dict_data_set[sub_key]['fname']
    fpath_data = os.path.join(dir_data, fname_data)
    key_lat = dict_data_set[sub_key]['key_lat']
    key_lon = dict_data_set[sub_key]['key_lon']
    key_y = dict_data_set[sub_key]['key_y']
    return(fpath_data, key_lat, key_lon, key_y)

def get_data(
    fpath
):
    msg_str = msg_str0.format(fpath)
    print(msg_str)
    data = pd.read_csv(fpath, header=0)
    msg_str = msg_str3.format(data.shape)
    print(msg_str)
    msg_str = msg_str5.format(data.columns)
    print(msg_str)
    return(data)

def centre_data(
    data, length, key_lat, key_lon,
    lat_centre, lon_centre
):
    """
    Subtract the centre coordinates of the region
    and then see whether the "centred coordinates"
    are within the region specified by length.
    """
    print('key_lat: {}'.format(key_lat))
    print('length: {}'.format(length))
    msg_str = msg_str1.format(data[key_lat].min(), data[key_lat].max())
    data = data[np.abs(data[key_lat] + lat_centre) < length]
    data = data[np.abs(data[key_lon] - lon_centre) < length]
    msg_str = msg_str2.format(data[key_lat].min(), data[key_lat].max())
    print(msg_str)
    msg_str = msg_str3.format(data.shape)
    print(msg_str)
    return(data)

def run_gp(
    data, key_lat, key_lon, key_y
):
    print('Running GP')
    X = np.array([data[key_lat], data[key_lon]]).T
    Y = np.array([data[key_y]]).T
    kernel = GPy.kern.Matern32(2)
    msg_str = msg_str4.format(X.shape, Y.shape)
    model = GPy.models.GPRegression(X, Y, kernel)
    model.optimize(messages=True)
    fig = plt.figure(figsize = (10, 10))
    f = model.plot()
    print(model)
    return(X, Y, model)

# Dataset1: grav
This seems pretty weird -- the gravity data seems to have a very long length scale and no obvious noise.  But we can see from the contours that there is some structure.  Not sure what to make of that.

In [ ]:
dict_data_set = dict_data_set1
sub_key = 'grav'
lt_val = length_grav

In [ ]:
fpath_data, key_lat, key_lon, key_y = get_vars(dict_data_set, sub_key)
data = get_data(fpath_data)

In [ ]:
data

In [ ]:
data_centred = centre_data(
    data, lt_val, 
    key_lat, key_lon,
    lat_centre, lon_centre
)

In [ ]:
data_centred['Latitude'].max(),data_centred['Latitude'].min()

In [ ]:
data_centred['Longitude'].max(),data_centred['Longitude'].min()

In [ ]:
X, Y, model = run_gp(data_centred, key_lat, key_lon, key_y)

In [ ]:
f = model.plot()
plt.savefig('/Users/davidkohn/Desktop/dataset1-grav.png')

# Dataset1: mag
Magnetism, on the other hand, has at least some non-zero Gaussian noise to it.  But surely the length scale is kind of out of whack?  And are those repeated points there?

In [ ]:
dict_data_set = dict_data_set1
sub_key = 'mag'
lt_val = length_mag

In [ ]:
fpath_data, key_lat, key_lon, key_y = get_vars(dict_data_set, sub_key)
data = get_data(fpath_data)

In [ ]:
data_centred = centre_data(data, lt_val, key_lat, key_lon)

Oooh looks like they are.  Well, in a way that's useful, if those are real -- in principle they give us the noise scale.  But if it isn't real, it's not clear this would have worked.

In [ ]:
X, Y, model = run_gp(data_centred, key_lat, key_lon, key_y)

In [ ]:
f = model.plot()
plt.savefig('/Users/davidkohn/Desktop/dataset1-mag.png')

In [ ]:
# looking at mag X, Y
dX0 = X[:,0].reshape(36,36)[:,0]
print(dX0)
print(dX0[1:] - dX0[:-1])
dX1 = X[:,1].reshape(36,36)[1,:]
print(dX1)
print(dX1[1:] - dX1[:-1])

# Dataset2: grav_north

In [ ]:
dict_data_set = dict_data_set2
sub_key = 'grav_north'
length = length_grav

In [ ]:
fpath_data, key_lat, key_lon, key_y = get_vars(dict_data_set, sub_key)
data = get_data(fpath_data)

In [ ]:
data['lat'] = 0
data['lon'] = 0
for idx, (x, y) in enumerate(zip(data['X'], data['Y'])):
    lat, lon = pyproj.transform(mgaz50, gda94, x, y)
    data.loc[idx, 'lat'] = lat
    data.loc[idx, 'lon'] = lon

In [ ]:
data_new = centre_data(data, length, 'lon', 'lat', lat_centre, lon_centre)

In [ ]:
data_new['GASCOYNE_NORTH_1'] = data_new['GASCOYNE_NORTH_1'] - data_new['GASCOYNE_NORTH_1'].mean()

In [ ]:
X, Y, model = run_gp(data_new, 'lon', 'lat', key_y)

In [ ]:
f = model.plot()
plt.savefig('/Users/davidkohn/Desktop/dataset2-grav-north.png')

# Dataset2: grav_south

In [ ]:
dict_data_set = dict_data_set2
sub_key = 'grav_south'
length = length_grav

In [ ]:
fpath_data, key_lat, key_lon, key_y = get_vars(dict_data_set, sub_key)
data = get_data(fpath_data)

In [ ]:
data['lat'] = 0
data['lon'] = 0
for idx, (x, y) in enumerate(zip(data['X'], data['Y'])):
    lat, lon = pyproj.transform(mgaz50, gda94, x, y)
    data.loc[idx, 'lat'] = lat
    data.loc[idx, 'lon'] = lon

In [ ]:
data_new = centre_data(data, length, 'lat', 'lon')

In [ ]:
X, Y = run_gp(data_new, 'lon', 'lat', key_y)

# Dataset2: mag

In [ ]:
import decimal

In [ ]:
dict_data_set = dict_data_set2
sub_key = 'mag'
length = length_mag_metres

In [ ]:
fpath_data, key_lat, key_lon, key_y = get_vars(dict_data_set, sub_key)

data = pd.read_csv(
    fpath_data,
    #converters={'MAG_PD': decimal.Decimal}
)

In [ ]:
data_centred = centre_data2(
    data, length, 
    'Y', 'X',
    northings_centre, eastings_centre, 
)

In [ ]:
data_centred.reset_index(inplace = True)
mag_pd = data_centred['MAG_PD']
new_mag_pd = mag_pd.astype('float64')
new_mag_pd = (new_mag_pd - new_mag_pd.mean())
data_centred['MAG_PD'] = new_mag_pd

In [ ]:
data_centred['lat'] = 0
data_centred['lon'] = 0
for idx, (x, y) in enumerate(zip(data_centred['X'], data_centred['Y'])):
    lat, lon = pyproj.transform(mgaz50, gda94, x, y)
    data_centred.loc[idx, 'lat'] = lat
    data_centred.loc[idx, 'lon'] = lon
    if idx % 100000 == 0: print(idx)

In [ ]:
X, Y, model = run_gp(data_centred, 'lon', 'lat', key_y)

In [ ]:
f = model.plot()
plt.savefig('/Users/davidkohn/Desktop/dataset2-mag.png')

In [ ]:
def centre_data2(
    data, length, key_lat, key_lon,
    lat_centre, lon_centre
):
    """
    Subtract the centre coordinates of the region
    and then see whether the "centred coordinates"
    are within the region specified by length.
    """
    print('key_lat: {}'.format(key_lat))
    print('length: {}'.format(length))
    msg_str = msg_str1.format(data[key_lat].min(), data[key_lat].max())
    data = data[np.abs(data[key_lat] - lat_centre) < length]
    data = data[np.abs(data[key_lon] - lon_centre) < length]
    msg_str = msg_str2.format(data[key_lat].min(), data[key_lat].max())
    print(msg_str)
    msg_str = msg_str3.format(data.shape)
    print(msg_str)
    return(data)

In [ ]:
d1 = get_data('/Users/davidkohn/dev/obsidian/data/dataset1/mag_TMI_gascoyne.txt')

In [ ]:
d1